# Transfer Learning with Pre-trained Models on CIFAR-100

## Introduction
This notebook demonstrates the use of transfer learning with various pre-trained models to classify images from the CIFAR-100 dataset. We will leverage architectures such as ResNet50, VGG16, and MobileNetV2 that have been pre-trained on ImageNet.

## 1. Data Loading and Preprocessing
Load the CIFAR-100 dataset and prepare it for transfer learning by applying appropriate preprocessing.

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet50
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_vgg16
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as preprocess_mobilenetv2

(X_train, y_train), (X_test, y_test) = cifar100.load_data()

X_train_resnet50 = preprocess_resnet50(X_train)
X_test_resnet50 = preprocess_resnet50(X_test)

X_train_vgg16 = preprocess_vgg16(X_train)
X_test_vgg16 = preprocess_vgg16(X_test)

X_train_mobilenetv2 = preprocess_mobilenetv2(X_train)
X_test_mobilenetv2 = preprocess_mobilenetv2(X_test)

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


## 2. Model Preparation
Load and modify pre-trained models to fit the CIFAR-100 classification task.

### 2.1 Using ResNet50

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load pre-trained ResNet50 model without the top layer
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

x = GlobalAveragePooling2D()(base_model_resnet50.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(100, activation='softmax')(x)

model_resnet50 = Model(inputs=base_model_resnet50.input, outputs=predictions)

for layer in base_model_resnet50.layers:
    layer.trainable = False

model_resnet50.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### 2.2 Using VGG16

In [ ]:
from tensorflow.keras.applications import VGG16

# Load pre-trained VGG16 model without the top layer
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
# Add new layers
x = GlobalAveragePooling2D()(base_model_vgg16.output)
x = Dense(512, activation='relu')(x)
predictions = Dense(100, activation='softmax')(x)
model_vgg16 = Model(inputs=base_model_vgg16.input, outputs=predictions)

for layer in base_model_vgg16.layers:
    layer.trainable = False

model_vgg16.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


### 2.3 Using MobileNetV2

In [ ]:
from tensorflow.keras.applications import MobileNetV2

# Load pre-trained MobileNetV2 model without the top layer
base_model_mobilenetv2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
# Add new layers
x = GlobalAveragePooling2D()(base_model_mobilenetv2.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(100, activation='softmax')(x)
model_mobilenetv2 = Model(inputs=base_model_mobilenetv2.input, outputs=predictions)

for layer in base_model_mobilenetv2.layers:
    layer.trainable = False

model_mobilenetv2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

<ipython-input-5-e5714e7ffe5d>:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model_mobilenetv2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## 3. Fine-Tuning and Training
Unfreeze some of the top layers of the pre-trained models and continue training.

In [ ]:
# Example of unfreezing top layers of ResNet50
for layer in model_resnet50.layers[:143]:
    layer.trainable = False

for layer in model_resnet50.layers[143:]:
    layer.trainable = True

history_resnet50 = model_resnet50.fit(X_train_resnet50, y_train, epochs=10, validation_data=(X_test_resnet50, y_test))

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1762s 1s/step - accuracy: 0.2766 - loss: 3.1240 - val_accuracy: 0.3964 - val_loss: 2.3867
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1734s 1s/step - accuracy: 0.4549 - loss: 2.0628 - val_accuracy: 0.4090 - val_loss: 2.3536
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1756s 1s/step - accuracy: 0.5413 - loss: 1.6667 - val_accuracy: 0.4465 - val_loss: 2.2915
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1757s 1s/step - accuracy: 0.6180 - loss: 1.3439 - val_accuracy: 0.4525 - val_loss: 2.3533
Epoch 5/10
 126/1563 ━━━━━━━━━━━━━━━━━━━━ 25:26 1s/step - accuracy: 0.7167 - loss: 0.9616

## 4. Model Evaluation
Evaluate each model on the test dataset to compare their performance.

In [ ]:
acc_resnet50 = model_resnet50.evaluate(X_test_resnet50, y_test)[1]
acc_vgg16 = model_vgg16.evaluate(X_test_vgg16, y_test)[1]
acc_mobilenetv2 = model_mobilenetv2.evaluate(X_test_mobilenetv2, y_test)[1]

print(f'ResNet50 Accuracy: {acc_resnet50:.2f}')
print(f'VGG16 Accuracy: {acc_vgg16:.2f}')
print(f'MobileNetV2 Accuracy: {acc_mobilenetv2:.2f}')

## 5. Conclusion
Summarize the findings, discuss the effectiveness of transfer learning for this task, and suggest further improvements or experiments.